In [1]:
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt
from utilitiesDOS import read_dump, calculate_fft, calculate_autocorr

In [2]:
import os, sys, git
sys.path.append('pyplot-perso/')
from functions import complete_panel, save_figure, set_boundaries, \
    add_subplotlabels, set_boundaries, prepare_figure
from color_series1 import colors
path_figures = "/figures/"

In [3]:
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path)

### Import LAMMPS velocities

In [4]:
path_to_file = "LAMMPS-input/dump.lammpsvel"
velocities, n_atoms, n_frames, n_columns, printing_period, masses = read_dump(path_to_file)
# Here velocities has units of A/fs
print("Shape of the velocities array =", np.shape(velocities))
print("Number of frames =", n_frames)
print("Number of particles =", n_atoms)

Shape of the velocities array = (850, 3, 20001)
Number of frames = 20001
Number of particles = 850


### Choose correlation length

In [6]:
correlation_len = n_frames // 4
print("Correlation length =", correlation_len)

Correlation length = 5000


### Set timestep

In [7]:
dt = 2.32 # fs
min_dt = dt * printing_period # duration between two printed velocity
print("The velocities was printed every", min_dt, "fs")

The velocities was printed every 11.6 fs


### Calculate VACF

Should be weighted by mass

In [11]:
vacf_xyz = np.zeros((3, correlation_len), dtype=float)
vacf = np.zeros((correlation_len), dtype=float)
N_split = 0
for i_time in range(correlation_len, n_frames, correlation_len):
    for i_atom, m in zip(range(n_atoms), masses):
        for i_dim in range(3):
            vacf_xyz[i_dim] += m * calculate_autocorr(velocities[i_atom,
                                                             i_dim,i_time-correlation_len:i_time],
                                                   correlation_len)
    N_split += 1
for vxyz in vacf_xyz: # Summ all 3 directions x y and z
    vacf += vxyz
vxyz /= N_split
#vacf /= vacf[0] # Normalize the velocity correlation
time = np.arange(len(vacf)) * min_dt # fs
time /= 1000 # ps

In [10]:
for i_time in range(correlation_len, n_frames, correlation_len):
    print(i_time)

5000
10000
15000
20000


In [ ]:
filename = "vacf"
desired_transparency = True
for mode, mygray in zip(['light', 'dark'], [colors["mylightgray"], colors["mydarkgray"]]):    
    fig = prepare_figure(mode, transparency=desired_transparency, desired_figsize=(18,6))
    ax, n, l_tot, c_tot = [], 0, 1, 1
    n += 1
    ax.append(plt.subplot(l_tot, c_tot, n))
    ax[-1].semilogx(time, vacf, 'o-', color=colors["mycyan"],
                markersize = 12, linewidth=2)
    complete_panel(ax[-1], r'$t$ (ps)', r'$\textrm{VACF}$',
                   legend=False, axis_color=mygray, xpad=15, locator_x=None)
    set_boundaries(plt, x_boundaries=(0.008, 1), y_boundaries=(-0.2, 1.2)) 
    save_figure(plt, fig, mode, git_path, path_figures, filename, transparency = desired_transparency)

### Calculate DOS

In [ ]:
freq, dos = calculate_fft(time, vacf).T

In [ ]:
filename = "dos"
desired_transparency = True
for mode, mygray in zip(['light', 'dark'], [colors["mylightgray"], colors["mydarkgray"]]):    
    fig = prepare_figure(mode, transparency=desired_transparency, desired_figsize=(18,6))
    ax, n, l_tot, c_tot = [], 0, 1, 1
    n += 1
    ax.append(plt.subplot(l_tot, c_tot, n))
    ax[-1].semilogx(freq, dos*1000, 'o-', color=colors["mycyan"],
                markersize = 12, linewidth=2)
    complete_panel(ax[-1], r'$f$ (MHz)', r'$\textrm{DoS}$ (ns)',
                   legend=False, axis_color=mygray, xpad=15, locator_x=None)
    # set_boundaries(plt, x_boundaries=(8, 1000), y_boundaries=(-0.2, 1.2)) 
    save_figure(plt, fig, mode, git_path, path_figures, filename, transparency = desired_transparency)